In [1]:
import numpy as np
import itertools
import picos as pic
import Fun as f
import mosek
import sys

import collections

In [84]:
def SDP(n,K,d,D):
    
    ##### SDP #####

    pic.available_solvers()
    
    Ap = pic.RealVariable("Ap", n+1)  # A column vector with n+1 elements.
    
    sdp = pic.Problem()
    
    #sdp.options["abs_*_tol"] = 1e-05
    
    sdp.set_objective('find') 
                   
    ##### Adding matrix constraints #####
    
    sdp.add_constraint(Ap[0]==1)
    
    eps=1
    
    ##### NPA #####
        
    ### Constructing Beta matrices ###

    Bmatrix=f.BetaMatrices(Ap,n) #### B_j(x)>>0 ####
    
    sdp.add_list_of_constraints(pic.block(Bmatrix[k])>>0 for k in range(0,int(n/2)+1))
    
    ##### Adding K-L constrains #####
    
    ### K*B'_j-A'_j=0  for d > j >= 0 ###
    
    sdp.add_list_of_constraints((K*Ap[n-i]-Ap[i])/eps== 0 for i in range(d)) 
    
    ### A_j>=0 ###
    
    for m in range(n+1):
        A=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*Ap[j] for j in range(m+1))
        sdp.add_constraint(A/eps >= 0)

    ### K*B_j-A_j>=0 ###
    
    for m in range(n+1):   
                                    
        KBA=sum((-1)**(m-j)*D**(j)*f.comb(n-j,n-m)*(K*Ap[n-j]-Ap[j]) for j in range(m+1))
        sdp.add_constraint(KBA/eps >= 0)
    
    ##### Adding Shadow constrains #####
    
    ### sum(K_j*A'_j)>=0 ###
    
    for j in range(n+1):
        
        S=sum(f.K_fun(n-j,i,n)*Ap[i] for i in range(n+1))
        #sdp.add_constraint(S/eps >= 0)
        
    ### Convention for K=1 codes ###
    
    if K==1:
        
        sdp.add_list_of_constraints(Ap[i]== f.comb(n,i)/D**i for i in range(1,d))
    
    sol = sdp.solve(primals=False)
    
    task=sdp.strategy.solver.int

    return task

    """
    if sol.problemStatus == "feasible":
        
        return True    
    
    elif sol.problemStatus == "infeasible":
        task=sdp.strategy.solver.int
        task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
        print(task.solutionsummary(mosek.streamtype.msg))
        
        return False
    else:
        task=sdp.strategy.solver.int
        task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
        print(task.solutionsummary(mosek.streamtype.msg))
          
        return sol.problemStatus
    """

Codes=[
[1,0,1],[1,1,1],
[2,0,2],[2,1,1],[2,2,1],
[3,0,2],[3,1,1],[3,2,1],[3,3,1],
[4,0,2],[4,1,2],[4,2,2],[4,3,1],[4,4,1],
[5,0,3],[5,1,3],[5,2,2],[5,3,1],[5,4,1],[5,5,1],
[6,0,4],[6,1,3],[6,2,2],[6,3,2],[6,4,2],[6,5,1],[6,6,1],
[7,0,3],[7,1,3],[7,2,2],[7,3,2],[7,4,2],[7,5,1],[7,6,1],[7,7,1],

[8,0,4],[8,1,3],[8,2,3],[8,3,3],[8,4,2],[8,5,2],[8,6,2],[8,7,1],[8,8,1],
[9,0,4],[9,1,3],[9,2,3],[9,3,3],[9,4,2],[9,5,2],[9,6,2],[9,7,1],[9,8,1],[9,9,1],
[10,0,4],[10,1,4],[10,2,4],[10,3,3],[10,4,3],[10,5,2],[10,6,2],[10,7,2],[10,8,2],[10,9,1],[10,10,1]
]

"""
#### Existing qubit codes ####

for code in Codes:
    
    n,K,d=code
    
    check=SDP(n,2**K,d,D)
    
    if check != True:
        print("%s=%s" % (code,check))

print("Done")




#### Not existing qubit codes (distance+1) ####

for code in Codes:
    
    n,K,d=code
    
    check=SDP(n,2**K,d+1,D)
    
    if check != False:
        print("[%s,%s,%s]=%s" % (code[0],code[1],code[2]+1,check))

print("Done")

"""

#### The program prints True if the sdp is feasible and false if it is infeasible ####

'\n#### Existing qubit codes ####\n\nfor code in Codes:\n    \n    n,K,d=code\n    \n    check=SDP(n,2**K,d,D)\n    \n    if check != True:\n        print("%s=%s" % (code,check))\n\nprint("Done")\n\n\n\n\n#### Not existing qubit codes (distance+1) ####\n\nfor code in Codes:\n    \n    n,K,d=code\n    \n    check=SDP(n,2**K,d+1,D)\n    \n    if check != False:\n        print("[%s,%s,%s]=%s" % (code[0],code[1],code[2]+1,check))\n\nprint("Done")\n\n'

In [36]:
D=2

Codes=[[10,24,3], [13,40,4], [21,7168,4], [24,49152,4], [22,384,6], [22,56,7], [24,24,8], [39,24,13]] 

#Codes=[[2,1,1]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

 

[10, 24, 3]=True
[13, 40, 4]=True

Interior-point solution summary
  Problem status  : UNKNOWN
  Solution status : UNKNOWN
  Primal.  obj: 0.0000000000e+00    nrm: 3e+09    Viol.  con: 1e+04    var: 0e+00    barvar: 0e+00  
  Dual.    obj: 3.8380878665e-05    nrm: 7e+00    Viol.  con: 9e-19    var: 9e-06    barvar: 8e-07  
None
[21, 7168, 4]=unknown

Interior-point solution summary
  Problem status  : UNKNOWN
  Solution status : UNKNOWN
  Primal.  obj: 0.0000000000e+00    nrm: 2e+11    Viol.  con: 2e+06    var: 0e+00    barvar: 0e+00  
  Dual.    obj: 1.8176054873e-04    nrm: 4e+01    Viol.  con: 5e-18    var: 8e-06    barvar: 2e-06  
None
[24, 49152, 4]=unknown

Interior-point solution summary
  Problem status  : UNKNOWN
  Solution status : UNKNOWN
  Primal.  obj: 0.0000000000e+00    nrm: 3e+08    Viol.  con: 3e+03    var: 0e+00    barvar: 0e+00  
  Dual.    obj: 1.1917197269e-04    nrm: 4e+01    Viol.  con: 3e-18    var: 3e-06    barvar: 2e-06  
None
[22, 384, 6]=unknown

Interior-po

In [15]:
Codes=[
[16,5,5],
[17,3,6],[17,5,5],[17,6,5],
[18,3,6],[18,4,6],
[19,3,6],[19,4,6],[19,5,6],[19,7,5],[19,10,4],
[20,2,7],[20,3,7],[20,5,6],[20,6,6],[20,7,5],[20,8,5],[20,11,4]  
]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],2**codes[1],codes[2],D)))

[16, 5, 5]=True
[17, 3, 6]=True
[17, 5, 5]=True
[17, 6, 5]=True
[18, 3, 6]=True
[18, 4, 6]=True
[19, 3, 6]=True
[19, 4, 6]=True
[19, 5, 6]=True
[19, 7, 5]=True
[19, 10, 4]=True
[20, 2, 7]=True
[20, 3, 7]=True
[20, 5, 6]=True
[20, 6, 6]=True
[20, 7, 5]=True
[20, 8, 5]=True
[20, 11, 4]=True


In [6]:
#### Non-additive ####

D=2

Codes=[[7,26,2],[9,112,2],[11,460,2],[13,1877,2],[15,7606,2],
      [7,3,3],[8,9,3],[9,13,3],[11,53,3],[12,89,3],[13,204,3],[14,324,3],[15,580,3],
      [7,1,4],[10,5,4],[11,7,4],[12,20,4],[13,40,4],[14,102,4],[15,150,4],
      [13,3,5],[14,10,5],[15,18,5]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],codes[1],codes[2],D)))

[7, 26, 2]=True
[9, 112, 2]=True
[11, 460, 2]=True
[13, 1877, 2]=True
[15, 7606, 2]=True
[7, 3, 3]=True
[8, 9, 3]=True
[9, 13, 3]=True
[11, 53, 3]=True
[12, 89, 3]=True
[13, 204, 3]=True
[14, 324, 3]=True
[15, 580, 3]=True
[7, 1, 4]=True
[10, 5, 4]=True
[11, 7, 4]=True
[12, 20, 4]=True
[13, 40, 4]=True
[14, 102, 4]=True
[15, 150, 4]=True
[13, 3, 5]=True
[14, 10, 5]=True
[15, 18, 5]=True


In [72]:
Codes=[[7, 0, 4], [15, 4, 5], [15, 7, 4], [16, 8, 4], [19, 3, 3], [22, 14, 4], [25, 0, 10]]

Codes=[[25, 0, 10]]

for codes in Codes:
    print("%s=%s" % (codes,SDP(codes[0],2**codes[1],codes[2],D)))


Interior-point solution summary
  Problem status  : UNKNOWN
  Solution status : UNKNOWN
  Primal.  obj: 0.0000000000e+00    nrm: 6e+06    Viol.  con: 3e+05    var: 0e+00    barvar: 0e+00  
  Dual.    obj: 4.9897264601e-03    nrm: 1e+05    Viol.  con: 6e-17    var: 2e-05    barvar: 3e-05  
None
[25, 0, 10]=unknown


In [85]:
D=2

#codes=[25, 0, 10]

codes=[7, 0, 4]

task=SDP(codes[0],D**codes[1],codes[2],D)

In [91]:
task.set_Stream(mosek.streamtype.msg, sys.stdout.write)

task.solutionsummary(mosek.streamtype.msg)

#task.set_Stream(mosek.streamtype.msg, sys.stdout.write)
    
#task.analyzesolution(mosek.streamtype.msg,mosek.soltype.itg)

task.analyzeproblem(mosek.streamtype.msg)  



Interior-point solution summary
  Problem status  : PRIMAL_AND_DUAL_FEASIBLE
  Solution status : OPTIMAL
  Primal.  obj: 0.0000000000e+00    nrm: 4e+01    Viol.  con: 6e-14    var: 0e+00    barvar: 3e-15  
  Dual.    obj: -8.0491169285e-15   nrm: 4e+01    Viol.  con: 0e+00    var: 7e-15    barvar: 2e-12  

                    Analyzing the problem

*** Structural report
                      Dimensions
                      Constraints   Variables     Matrix var.   Cones         ACC           DJC         
                      94            8             4             0             0             0

                      Constraint and bound types
                      Free          Lower         Upper         Ranged        Fixed       
        Constraints:  0             16            0             0             78          
          Variables:  8             0             0             0             0           

                      Constraint matrix for the scalarized problem
   

In [92]:
task.set_Stream(mosek.streamtype.msg, sys.stdout.write)

task.putintparam(mosek.iparam.log, 2)

task.optimize()

#task.solutionsummary(mosek.streamtype.msg)
#task.analyzeproblem(mosek.streamtype.msg) 

#task.analyzesolution(mosek.streamtype.msg,mosek.soltype.itg)

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 94              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 8               
  Matrix variables       : 4               
  Integer variables      : 0               

Optimizer started.
Optimizer terminated. Time: 0.00    



Error: rescode.err_undef_solution(22000): The solution with code 2 is not defined.